In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings('ignore')

In [ ]:
DATA_PATH = 'data_phase2/Q1_data.csv'
crypto_data = pd.read_csv(DATA_PATH)

In [ ]:
crypto_data.head()

In [ ]:
crypto_data.info()

In [ ]:
crypto_data.describe()

In [ ]:
# Plot Scatter Plot of Market Cap vs. Volume
fig = px.scatter(crypto_data, x="market_cap", y="volume", color="symbol")
fig.update_layout(
    title='Market Cap vs. Volume',
    title_x=0.5,
    xaxis_title='Market Cap ($)',
    yaxis_title='Volume ($)',    
)
fig.show()

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
بخش اول
</font>

In [ ]:
# Run K-Means Clustering (with k = 5) on the dataset using the market cap and volume features
kmeans = KMeans(n_clusters=5, init='k-means++', random_state=42)
kmeans.fit(crypto_data[['market_cap', 'volume']])
crypto_data['cluster_kmeans'] = [f'cluster {label + 1}' for label in kmeans.labels_]

In [ ]:
# Plot Scatter Plot of Market Cap vs. Volume, Clussified by K-Means Clustering (k = 5)
fig = go.Figure()
for cluster in crypto_data['cluster_kmeans'].sort_values().unique():
    fig.add_trace(
        go.Scatter(
            x=crypto_data[crypto_data['cluster_kmeans'] == cluster]['market_cap'], 
            y=crypto_data[crypto_data['cluster_kmeans'] == cluster]['volume'],
            mode='markers', 
            name=cluster,
            marker=dict(opacity=0.7)))

centers = kmeans.cluster_centers_
fig.add_trace(go.Scatter(
    x=centers[:, 0],
    y=centers[:, 1],
    mode='markers',
    marker=dict(color='Black', size=6, line=dict(color='Black', width=2)),
    name='Centers'
))

fig.update_layout(
    title='Market Cap vs. Volume, Clussified by K-Means Clustering (k = 5)',
    title_x=0.5,
    xaxis_title='Market Cap ($)',
    yaxis_title='Volume ($)',    
)

fig.show()

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
بخش دوم
</font>

In [ ]:
# Find the optimal number of clusters using the elbow method
wcss = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42)
    kmeans.fit(crypto_data[['market_cap', 'volume']])
    wcss.append(kmeans.inertia_)

fig = px.line(x=range(1, 11), y=wcss, markers='o')
fig.update_layout(
    title='Elbow Method',
    title_x=0.5,
    xaxis_title='Number of Clusters',
    yaxis_title='Within-Cluster Sum of Square',    
)
fig.update_xaxes(nticks=20)

In [ ]:
silhouette_scores = []
for k in range(2, 10):
    kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42)
    cluster_labels = kmeans.fit_predict(crypto_data[['market_cap', 'volume']])  # Replace 'data' with your dataset
    silhouette_avg = silhouette_score(crypto_data[['market_cap', 'volume']], cluster_labels)
    silhouette_scores.append(silhouette_avg)

best_k = silhouette_scores.index(max(silhouette_scores)) + 2
print(f'Best k: {best_k}')

In [ ]:
# Run K-Means Clustering again, this time for best k(=4) on the dataset using the market cap and volume features
kmeans_best = KMeans(n_clusters=best_k, init='k-means++', random_state=42)
kmeans_best.fit(crypto_data[['market_cap', 'volume']])
crypto_data['cluster_kmeans (best k)'] = [f'cluster {label + 1}' for label in kmeans_best.labels_]

# Plot Scatter Plot of Market Cap vs. Volume, Clussified by K-Means Clustering using best k(= 4)
fig = go.Figure()
for cluster in crypto_data['cluster_kmeans (best k)'].sort_values().unique():
    fig.add_trace(
        go.Scatter(
            x=crypto_data[crypto_data['cluster_kmeans (best k)'] == cluster]['market_cap'], 
            y=crypto_data[crypto_data['cluster_kmeans (best k)'] == cluster]['volume'],
            mode='markers', 
            name=cluster,
            marker=dict(opacity=0.7)))

centers = kmeans_best.cluster_centers_
fig.add_trace(go.Scatter(
    x=centers[:, 0],
    y=centers[:, 1],
    mode='markers',
    marker=dict(color='Black', size=6, line=dict(color='Black', width=2)),
    name='Centers'
))

fig.update_layout(
    title='Market Cap vs. Volume, Clussified by K-Means Clustering, for best k(= 4)',
    title_x=0.5,
    xaxis_title='Market Cap ($)',
    yaxis_title='Volume ($)',    
)

fig.show()

<p dir=rtl style="direction: rtl;text-align: center;line-height:200%;font-family:vazir;font-size:medium;color:#0099cc"><font face="vazir" size=3><i>
[تحلیل] برای بدست آوردن مقدار بهینه k از دو روش elbow  و silhouette بهره بردیم. همان طور که در نمودار elbow می توان مشاهده کرد،‌ اگر مقدار k را بزرگتر از ۴ قرار دهیم، تغییر قابل توجهی در مقدار خطای WCSS مشاهده نمی شود؛ بنابراین می توان نتیجه گرفت تعداد خوشه بهینه برای این مجموعه داده برابر با ۴ است و روش sihouette هم در ادامه این تعداد خوشه را تایید می کند. 
اما باید پرسید «آیا ۴ خوشه برای این مجموعه داده تفسیر پذیر است؟» در نگاه اول این طور به نظر می رسد،‌ چرا که داده های ما اصلا مربوط به ۴ نوع رمزارز هستند.  اما اگر دوباره الگوریتم را با k = 4 اجرا کنیم،‌ مشاهده می کنیم که الگوریتم داده های مربوط به دو رمزارز (BNB و USDT)  را در یک خوشه قرار می دهد و از طرف دیگر، داده های مربوط به رمز ارز بیت کوین را در دو خوشه قرار می دهد. ما ظاهرا،‌ در نگاهی ساده و اولیه، انتظار داشتیم که اصطلاحا الگوریتم داده ها را از مفصل هایش ببرد و داده های  مربوط به هر رمز ارز را در یک خوشه مجزا طبقه بندی کند. باید پرسید اینکه داده های بیت کوین در دو خوشه طبقه بندی می شوند چه تفسیری دارد؟ و این که چرا دو رمز ارز BNB و USDT در یک خوشه قرار می گیرند؟ شاید اگر یک یا چند بعد دیگر از به داده هایمان اضافه کنیم،‌ داده های مربوط به هر رمز ارز دقیقا در یک خوشه طبقه بندی شوند.

</i></font></p>

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
بخش سوم
</font>

In [ ]:
# Run DBSCAN Clustering on the dataset using the market cap and volume features
market_caps_scaled = crypto_data['market_cap'].values / 4
volumes = crypto_data['volume'].values
dbscan = DBSCAN(eps=0.5e10 + 0.7e9 + 0.5e8, min_samples=4)
dbscan.fit(np.stack((market_caps_scaled, volumes), axis=-1))
crypto_data['cluster_dbscan'] = [
    f'cluster {label + 1}' if label != -1 else 'noise' for label in dbscan.labels_
]
n_clusters = len(set(dbscan.labels_)) - (1 if -1 in dbscan.labels_ else 0)
n_noises = len([label for label in dbscan.labels_ if label == -1])
print(f'Number of clusters found: {n_clusters}')

print(f'Number of noises found: {n_noises}')

In [ ]:
# Plot Scatter Plot of Market Cap vs. Volume, Clussified by DBSCAN Clustering
fig = go.Figure()

clusters_colors = px.colors.qualitative.Plotly
for i, cluster in enumerate(crypto_data['cluster_dbscan'].unique()):
    if cluster == 'noise':
        cluster_color = 'red'
    else:
        cluster_color = clusters_colors[i + 4]
    fig.add_trace(
        go.Scatter(
            x=crypto_data[crypto_data['cluster_dbscan'] == cluster]['market_cap'], 
            y=crypto_data[crypto_data['cluster_dbscan'] == cluster]['volume'],
            mode='markers', 
            name=cluster,
            marker=dict(opacity=0.8, color=cluster_color)))

fig.update_layout(
    title='Market Cap vs. Volume, Clussified by DBSCAN Clustering',
    title_x=0.5,
    xaxis_title='Market Cap ($)',
    yaxis_title='Volume ($)',    
)

<p dir=rtl style="direction: rtl;text-align: center;line-height:200%;font-family:vazir;font-size:medium;color:#0099cc"><font face="vazir" size=3><i>
[تحلیل]  الگوریتم DBSCAN حساس به دو پارامتر min_samples  و پارامتر eps (شعاع) است و هر چه eps را کوچکتر در نظر بگیریم، خوشه‌های بیشتر و کوچکتری تشکیلی می‌شود. همچنین هر چه قدر min_samples را بزرگ‌تر در نظر بگیریم، احتمال ایجادِ خوشه‌ها، کمتر می‌شود زیرا الگوریتم باید تعداد نمونه‌های بیشتری را در یک شعاع خاص ببینید تا خوشه را تشکیل دهد.
</i></font></p>